Based on research by [MIT](https://www.science.org/doi/10.1126/scitranslmed.abb3652)

In [1]:
!pip install scipy

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
import pathlib
import scipy

In [3]:
path_to_training_dataset = "data/isic/train/"
data_dir_train = pathlib.Path(path_to_training_dataset)
print(data_dir_train)
path_to_test_dataset = "data/isic/test/"
data_dir_test = pathlib.Path(path_to_test_dataset)
print(data_dir_test)

data/isic/train
data/isic/test


In [4]:
# TODO: are these image count correct?

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4) # Splitting 20% for validation

train_generator = train_datagen.flow_from_directory(
    data_dir_train,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    data_dir_test,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 1348 images belonging to 9 classes.
Found 44 images belonging to 9 classes.


In [6]:
# TODO: experiment with increasing epochs. Graph accuracy and loss

# Baseline DCNN Model
def build_baseline_model():
    inputs = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    # outputs = Dense(6, activation='softmax')(x) # Assuming 6 classes
    outputs = Dense(9, activation='softmax')(x) # Adjusted to 9 classes
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

baseline_model = build_baseline_model()
baseline_model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
43/43 [==============================] - 14s 303ms/step - loss: 2.6209 - accuracy: 0.1721 - val_loss: 2.1962 - val_accuracy: 0.1364
Epoch 2/10
43/43 [==============================] - 11s 253ms/step - loss: 2.2114 - accuracy: 0.1818 - val_loss: 2.2440 - val_accuracy: 0.0909
Epoch 3/10
43/43 [==============================] - 11s 253ms/step - loss: 2.0616 - accuracy: 0.2151 - val_loss: 3.0426 - val_accuracy: 0.0909
Epoch 4/10
43/43 [==============================] - 11s 251ms/step - loss: 1.9536 - accuracy: 0.3027 - val_loss: 3.6725 - val_accuracy: 0.1136
Epoch 5/10
43/43 [==============================] - 11s 251ms/step - loss: 1.7944 - accuracy: 0.3880 - val_loss: 3.5708 - val_accuracy: 0.2273
Epoch 6/10
43/43 [==============================] - 11s 252ms/step - loss: 1.6902 - accuracy: 0.4228 - val_loss: 3.6010 - val_accuracy: 0.2727
Epoch 7/10
43/43 [==============================] - 11s 251ms/step - loss: 1.5852 - accuracy: 0.4614 - val_loss: 4.2112 - val_accuracy: 0.2727

In [8]:
# TODO: experiment with increasing epochs. Graph accuracy and loss

# Transfer Learning with VGG16
def build_vgg16_transfer_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    # outputs = Dense(6, activation='softmax')(x) # Assuming 6 classes
    outputs = Dense(9, activation='softmax')(x) # Adjusted to 9 classes
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg16_model = build_vgg16_transfer_model()
vgg16_model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
43/43 [==============================] - 14s 312ms/step - loss: 12.4844 - accuracy: 0.1736 - val_loss: 9.5742 - val_accuracy: 0.1364
Epoch 2/10
43/43 [==============================] - 11s 265ms/step - loss: 9.6305 - accuracy: 0.2151 - val_loss: 8.5066 - val_accuracy: 0.1364
Epoch 3/10
43/43 [==============================] - 11s 263ms/step - loss: 7.7775 - accuracy: 0.2263 - val_loss: 5.7745 - val_accuracy: 0.1364
Epoch 4/10
43/43 [==============================] - 12s 266ms/step - loss: 6.0539 - accuracy: 0.2930 - val_loss: 5.4103 - val_accuracy: 0.1364
Epoch 5/10
43/43 [==============================] - 11s 262ms/step - loss: 4.5032 - accuracy: 0.3242 - val_loss: 4.9361 - val_accuracy: 0.1364
Epoch 6/10
43/43 [==============================] - 11s 262ms/step - loss: 3.6569 - accuracy: 0.3227 - val_loss: 4.2879 - val_accuracy: 0.1364
Epoch 7/10
43/43 [==============================] - 11s 265ms/step - loss: 3.4217 - accuracy: 0.3516 - val_loss: 4.2382 - val_accuracy: 0.159